# 資料前處理(Label encoding、 One hot encoding)
這兩個編碼方式的目的是為了將類別 (categorical)或是文字(text)的資料轉換成數字，而讓程式能夠更好的去理解及運算。
> Label encoding : 把每個類別 mapping 到某個整數，不會增加新欄位-->有序

> One hot encoding : 為每個類別新增一個欄位，用 0/1 表示是否--->無序

![](images/Encoder.PNG)


## Encoding Categorical features (or label)
![](images/Encoding.PNG)


In [52]:
import pandas as pd
import numpy as np


In [10]:
df = pd.DataFrame({'blood':['A','B','AB','O','B'], 
                   'Y':['high','low','high','mid','mid'],
                   'Z':[np.nan,np.nan,-1196,72,83]});
df

,blood,Y,Z
0,A,high,NaN
1,B,low,NaN
2,AB,high,-1196.0
3,O,mid,72.0
4,B,mid,83.0


# 方法一：sklearn - label encoder + onehot encoder
>onehot encoder要用2D array，若維度所以要用reshape(-1,1)<br>
>onehot encoder要數字，若資料文文字要先用label encoder轉數字

In [15]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoded_Y = encoder.fit_transform(df['blood'])
print(encoded_Y)

[0 2 1 3 2]


In [17]:
df['blood']=encoded_Y
df

,blood,Y,Z
0,0,high,NaN
1,2,low,NaN
2,1,high,-1196.0
3,3,mid,72.0
4,2,mid,83.0


In [23]:
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder()
d = np.array(df['blood'])
d.shape
onehot_df = onehot.fit_transform(d.reshape(-1,1)).toarray()
onehot_df

array([[1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]])

## One hot encoding
One Hot encoding的編碼邏輯為將類別拆成多個行(column)，每個列中的數值由1、0替代，當某一列的資料存在的該行的類別則顯示1，反則顯示0。

然在指定column進行編碼的情形下，One hot encoding<b>無法直接對字串進行編碼，必須先透過Label encoding將字串以數字取代後再進行One hot encoding處理。</b>

> categorical_features = [0]: 表示欲在data上執行One hot encoding的index為0

> data_le: 為經過Label encoding編碼的資料(註:OneHotEncoder的輸入要為2-D array，而Label encoding為1-D array)


OneHotEncoder會轉出scipy.csr_matrix資料結構用.toarray()轉array
從結果可以知道，數字0的column 代表的是A、數字1的column 代表的是B，而數字2的column 代表的是AB。
除了轉換字串外，One hot encoding也可以轉換數字。在此處的data就不需要先經過Label encoding編碼

```python
# importing one hot encoder from sklearn 
# There are changes in OneHotEncoder class 
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer 
   
# creating one hot encoder object with categorical feature 0 
# indicating the first column 
columnTransformer = ColumnTransformer([('encoder', 
                                        OneHotEncoder(), 
                                        [0])], 
                                      remainder='passthrough') 
  
data = np.array(columnTransformer.fit_transform(data), dtype = str) 
```

In [30]:
# importing one hot encoder from sklearn 
# There are changes in OneHotEncoder class 
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer 

# creating one hot encoder object with categorical feature 0 
# indicating the first column 
columnTransformer = ColumnTransformer([('encoder', 
                                        OneHotEncoder(), 
                                        [0])], 
                                      remainder='passthrough') 

data = np.array(columnTransformer.fit_transform(df), dtype = str) 
data

array([['1.0', '0.0', '0.0', '0.0', 'high', 'nan'],
       ['0.0', '0.0', '1.0', '0.0', 'low', 'nan'],
       ['0.0', '1.0', '0.0', '0.0', 'high', '-1196.0'],
       ['0.0', '0.0', '0.0', '1.0', 'mid', '72.0'],
       ['0.0', '0.0', '1.0', '0.0', 'mid', '83.0']], dtype='<U7')

In [32]:
data_le = pd.DataFrame(data)
data_le

,0,1,2,3,4,5
0,1.0,0.0,0.0,0.0,high,nan
1,0.0,0.0,1.0,0.0,low,nan
2,0.0,1.0,0.0,0.0,high,-1196.0
3,0.0,0.0,0.0,1.0,mid,72.0
4,0.0,0.0,1.0,0.0,mid,83.0


# 方法二：Keras - label encoder + to_categorical
>to_categorical要數字，若資料文文字要先用label encoder轉數字

In [59]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

df = pd.DataFrame({'blood':['A','B','AB','O','B'], 
                   'Y':['high','low','high','mid','mid'],
                   'Z':[np.nan,np.nan,-1196,72,83]});
df
# label encoder 


# convert integers to one hot encoding




ImportError: To use Keras, you need to have `optree` installed. Install it via `pip install optree`

In [4]:
def to_categorical(y, num_classes=None, dtype='float32'):
    """Converts a class vector (integers) to binary class matrix.
    E.g. for use with categorical_crossentropy.
    # Arguments
        y: class vector to be converted into a matrix
            (integers from 0 to num_classes).
        num_classes: total number of classes.
        dtype: The data type expected by the input, as a string
            (`float32`, `float64`, `int32`...)
    # Returns
        A binary matrix representation of the input. The classes axis
        is placed last.
    # Example
    ```python
    # Consider an array of 5 labels out of a set of 3 classes {0, 1, 2}:
    > labels
    array([0, 2, 1, 2, 0])
    # `to_categorical` converts this into a matrix with as many
    # columns as there are classes. The number of rows
    # stays the same.
    > to_categorical(labels)
    array([[ 1.,  0.,  0.],
           [ 0.,  0.,  1.],
           [ 0.,  1.,  0.],
           [ 0.,  0.,  1.],
           [ 1.,  0.,  0.]], dtype=float32)
    ```
    """
    #將輸入y向量轉換為陣列
    y = np.array(y, dtype='int')
    #獲取陣列的行列大小
    input_shape = y.shape
    if input_shape and input_shape[-1] == 1 and len(input_shape) > 1:
        input_shape = tuple(input_shape[:-1])
    #y變為1維陣列
    y = y.ravel()
    #如果使用者沒有輸入分類個數，則自行計算分類個數
    if not num_classes:
        num_classes = np.max(y) + 1
    n = y.shape[0]
    #生成全為0的n行num_classes列的值全為0的矩陣
    categorical = np.zeros((n, num_classes), dtype=dtype)
    #np.arange(n)得到每個行的位置值，y裡邊則是每個列的位置值
    categorical[np.arange(n), y] = 1
    #進行reshape矯正
    output_shape = input_shape + (num_classes,)
    categorical = np.reshape(categorical, output_shape)
    return categorical

In [5]:
import numpy as np
import pandas as pd
country=['Taiwan','Australia','Ireland','Australia','Ireland','Taiwan']
age=[25,30,45,35,22,36]
salary=[20000,32000,59000,60000,43000,52000]
dic={'Country':country,'Age':age,'Salary':salary}
data=pd.DataFrame(dic)


from sklearn.preprocessing import LabelEncoder
import np_utils

# label encoder 
encoder = LabelEncoder()
encoded_Y = encoder.fit_transform(data['Country'])
print(encoded_Y)
data['Country']=encoded_Y
data

# convert integers to one hot encoding
dummy_y = to_categorical(encoded_Y)  # one-hot encoding [0. 1. 0.] 
dummy_y


ModuleNotFoundError: No module named 'np_utils'

In [14]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

# 創建示例數據
df = pd.DataFrame({'blood':['A','B','AB','O','B'], 
                   'Y':['high','low','high','mid','mid'],
                   'Z':[np.nan,np.nan,-1196,72,83]});
df

# label encoder 
encoder_Y = encoder.fit_transform(data['blood'])
print(encoder_Y)
df['blood']=encoder_Y
df

# convert integers to one hot encodingencoder = LabelEncoder()
integer_encoded = encoder.fit_transform(data['blood'])
keras_onehot = tf.keras.utils.to_categorical(integer_encoded)
encoder_Y

# 進行 one-hot 編碼
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
one_hot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(one_hot_encoded)
# print("Label Encoded:", integer_encoded)
# print("One-Hot Encoded:\n", one_hot_encoded)


[0 2 1 3 2]
[[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]


c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## 方法三：pd.get_dummies方法
![](images/Encoding_pd.PNG)
pd.get_dummies(df)
>get_dummies可以直接轉字串，反而無法轉換數字<br>
>get_dummies沒指定columns，會全部轉換

In [67]:
df = pd.DataFrame({'blood':['A','B','AB','O','B'], 
                   'Y':['high','low','high','mid','mid'],
                   'Z':[np.nan,np.nan,-1196,72,83]})
df1 = pd.get_dummies(df)
print(df1)
df2 = pd.get_dummies(df.blood)
print(df2)

        Z  blood_A  blood_AB  blood_B  blood_O  Y_high  Y_low  Y_mid
0     NaN        1         0        0        0       1      0      0
1     NaN        0         0        1        0       0      1      0
2 -1196.0        0         1        0        0       1      0      0
3    72.0        0         0        0        1       0      0      1
4    83.0        0         0        1        0       0      0      1
   A  AB  B  O
0  1   0  0  0
1  0   0  1  0
2  0   1  0  0
3  0   0  0  1
4  0   0  1  0


## 練習一：sklearn - label encoder + onehot encoder
下面的資料可以看到country那欄皆為字串， 大部分的模型都是基於數學運算，字串無法套入數學模型進行運算，<br>
在此先對其進行Label encoding編碼，我們從 sklearn library中導入 LabelEncoder class，對第一行資料進行fit及transform並取代之。

In [22]:
import numpy as np
import pandas as pd
country=['Taiwan','Australia','Ireland','Australia','Ireland','Taiwan']
age=[25,30,45,35,22,36]
salary=[20000,32000,59000,60000,43000,52000]
dic={'Country':country,'Age':age,'Salary':salary}
data=pd.DataFrame(dic)
data

,Country,Age,Salary
0,Taiwan,25,20000
1,Australia,30,32000
2,Ireland,45,59000
3,Australia,35,60000
4,Ireland,22,43000
5,Taiwan,36,52000


In [21]:
encoder = LabelEncoder()
encoded_Y = encoder.fit_transform(data['Country'])
print(encoded_Y)
data['Country']=encoded_Y
print(data)
onehot = OneHotEncoder()
d = np.array(data['Country'])
d.shape
onehot_data = onehot.fit_transform(d.reshape(-1,1)).toarray() # type: ignore
print(onehot_data)

[2 0 1 0 1 2]
   Country  Age  Salary
0        2   25   20000
1        0   30   32000
2        1   45   59000
3        0   35   60000
4        1   22   43000
5        2   36   52000
[[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [24]:
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer 
columnTransformer = ColumnTransformer([('encoder', 
                                        OneHotEncoder(), 
                                        [0])], 
                                      remainder='passthrough') 

data = np.array(columnTransformer.fit_transform(data), dtype = str) 
data

array([['0.0', '0.0', '1.0', '25.0', '20000.0'],
       ['1.0', '0.0', '0.0', '30.0', '32000.0'],
       ['0.0', '1.0', '0.0', '45.0', '59000.0'],
       ['1.0', '0.0', '0.0', '35.0', '60000.0'],
       ['0.0', '1.0', '0.0', '22.0', '43000.0'],
       ['0.0', '0.0', '1.0', '36.0', '52000.0']], dtype='<U32')

## 練習二：Keras - label encoder + to_categorical

In [25]:
from sklearn.preprocessing import LabelEncoder
# from keras.utils import np_utils

country=['Taiwan','Australia','Ireland','Australia','Ireland','Taiwan']
age=[25,30,45,35,22,36]
salary=[20000,32000,59000,60000,43000,52000]
dic={'Country':country,'Age':age,'Salary':salary}
data=pd.DataFrame(dic)
data

,Country,Age,Salary
0,Taiwan,25,20000
1,Australia,30,32000
2,Ireland,45,59000
3,Australia,35,60000
4,Ireland,22,43000
5,Taiwan,36,52000


In [30]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

encoder_Y = encoder.fit_transform(data['Country'])
print(encoder_Y)
data['Country']=encoder_Y
data

# convert integers to one hot encodingencoder = LabelEncoder()
integer_encoded = encoder.fit_transform(data['Country'])
keras_onehot = tf.keras.utils.to_categorical(integer_encoded)
encoder_Y

# 進行 one-hot 編碼
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
one_hot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(one_hot_encoded)

[2 0 1 0 1 2]
[[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## 練習三：Pandas.get_dummies
>　get_dummies : 僅能將字串轉換為One hot encoding表示形式， 沒指定columns會全部轉換。

In [39]:
country=['Taiwan','Australia','Ireland','Australia','Ireland','Taiwan']
age=[25,30,45,35,22,36]
salary=[20000,32000,59000,60000,43000,52000]
dic={'Country':country,'Age':age,'Salary':salary}
data=pd.DataFrame(dic)
data

,Country,Age,Salary
0,Taiwan,25,20000
1,Australia,30,32000
2,Ireland,45,59000
3,Australia,35,60000
4,Ireland,22,43000
5,Taiwan,36,52000


In [43]:
data1 = pd.get_dummies(data, dtype=int)
print(data1)
data2 = pd.get_dummies(data.Country, dtype=int)
print(data2)

   Age  Salary  Country_Australia  Country_Ireland  Country_Taiwan
0   25   20000                  0                0               1
1   30   32000                  1                0               0
2   45   59000                  0                1               0
3   35   60000                  1                0               0
4   22   43000                  0                1               0
5   36   52000                  0                0               1
   Australia  Ireland  Taiwan
0          0        0       1
1          1        0       0
2          0        1       0
3          1        0       0
4          0        1       0
5          0        0       1
